# Unificandos as amostras em um único dataset

Todas as amostras foram feitas com:
* Configuração de switch 22056  
* Tempo de switch de 2us 
* Intervalo de transmissão da tag de 100ms
* intervalo de transmissão do gateway de 200ms 


In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def pm(angle, deg = False):
    ''' Mantém o ângulo entre +- 180/pi '''
    factor = 180 if deg else np.pi

    if angle > factor:
        while angle > factor:
            angle -= 2*factor
    if angle < factor:
        while angle < - factor:
            angle += 2*factor
    return angle

pm = np.vectorize(pm)


def single_antenna_to_dataframe(read_fp: str, write_fp=None) -> pd.DataFrame:
    '''Transforma uma captura com uma única antena em um df '''
    raw =pd.read_json(read_fp)
    df= list()
    for group in raw[1]:
        df.append(group)
    df = pd.DataFrame(df)
    df = df.join(pd.DataFrame(df["aoa"].to_list()))
    df.pop("aoa")

    if write_fp is not None:
        df.to_json(write_fp)

    return df

In [3]:
name = os.listdir("tw")[0]
single_antenna_to_dataframe("tw/"+name)

,mac,timestamp,no,rssi,raw,frequency,iq
0,c300000a7a48,2024-10-04T17:03:27.558Z,0,-55,0201061bff3906cafb01140000ffef0102000000000000...,2402,"[94, -140, 123, 116, -137, 97, -71, -157, 158,..."
1,c300000a7a48,2024-10-04T17:03:28.091Z,0,-57,0201061bff3906cafb01140000ffe90100000000000000...,2426,"[-128, -79, 107, -107, 93, 114, -134, 59, -37,..."
2,c300000a7a48,2024-10-04T17:03:28.614Z,0,-57,0201061bff3906cafb01140000ffec0102000000000000...,2426,"[107, 103, -122, 86, -68, -138, 140, -44, 21, ..."
3,c300000a7a48,2024-10-04T17:03:29.148Z,0,-59,0201061bff3906cafb0114fffeffec0102000000000000...,2480,"[-3, 185, -180, -18, 42, -180, 166, 73, -107, ..."
4,c300000a7a48,2024-10-04T17:03:29.409Z,0,-59,0201061bff3906cafb0114feffecff0201000000000000...,2480,"[-175, -39, 79, -163, 159, 106, -114, 147, -11..."
...,...,...,...,...,...,...,...
228,c300000a7a48,2024-10-04T17:05:24.358Z,0,-56,0201061bff3906cafb01140000ecff0001000000000000...,2402,"[-47, 171, -145, -56, 99, -123, 115, 139, -143..."
229,c300000a7a48,2024-10-04T17:05:24.882Z,0,-56,0201061bff3906cafb0114feffecfffd00000000000000...,2402,"[148, 65, -99, 127, -114, -122, 139, -90, 67, ..."
230,c300000a7a48,2024-10-04T17:05:25.667Z,0,-56,0201061bff3906cafb01140000ffe90102000000000000...,2426,"[-142, -72, 101, -125, 115, 114, -127, 90, -59..."
231,c300000a7a48,2024-10-04T17:05:25.928Z,0,-57,0201061bff3906cafb01140000e9ff0201000000000000...,2426,"[-157, -15, 40, -160, 153, 55, -74, 145, -120,..."


# TW
Capturas no poli foram feitas a distância constante com o centro dos ângulos **na antena de referência**

In [4]:
tw= pd.DataFrame()

for file in os.listdir("tw"):
    df = single_antenna_to_dataframe("./tw/"+file)
    df["angle"] = file.split("-")[0].split("_")[-1]
    df["distance"] = file.split("-")[1].split("_")[-1].split(".")[0]
    tw = pd.concat((tw,df))
tw["capture"] = "tw"
tw

,mac,timestamp,no,rssi,raw,frequency,iq,angle,distance,capture
0,c300000a7a48,2024-10-04T17:03:27.558Z,0,-55,0201061bff3906cafb01140000ffef0102000000000000...,2402,"[94, -140, 123, 116, -137, 97, -71, -157, 158,...",0,100,tw
1,c300000a7a48,2024-10-04T17:03:28.091Z,0,-57,0201061bff3906cafb01140000ffe90100000000000000...,2426,"[-128, -79, 107, -107, 93, 114, -134, 59, -37,...",0,100,tw
2,c300000a7a48,2024-10-04T17:03:28.614Z,0,-57,0201061bff3906cafb01140000ffec0102000000000000...,2426,"[107, 103, -122, 86, -68, -138, 140, -44, 21, ...",0,100,tw
3,c300000a7a48,2024-10-04T17:03:29.148Z,0,-59,0201061bff3906cafb0114fffeffec0102000000000000...,2480,"[-3, 185, -180, -18, 42, -180, 166, 73, -107, ...",0,100,tw
4,c300000a7a48,2024-10-04T17:03:29.409Z,0,-59,0201061bff3906cafb0114feffecff0201000000000000...,2480,"[-175, -39, 79, -163, 159, 106, -114, 147, -11...",0,100,tw
...,...,...,...,...,...,...,...,...,...,...
210,c300000a7a63,2024-10-04T15:47:50.610Z,0,-52,0201061bff3906cafb0114fff9ffe40102000000000000...,2426,"[-16, -157, 151, 21, -55, 150, -139, -70, 95, ...",90,200,tw
211,c300000a7a63,2024-10-04T15:47:51.134Z,0,-55,0201061bff3906cafb0114fff6ffe20100000000000000...,2480,"[-37, -174, 165, -7, -47, 163, -155, -80, 106,...",90,200,tw
212,c300000a7a63,2024-10-04T15:47:51.671Z,0,-55,0201061bff3906cafb0114fff6ffe400fd000000000000...,2480,"[-163, -70, 97, -141, 92, 134, -160, 70, -36, ...",90,200,tw
213,c300000a7a63,2024-10-04T15:47:52.197Z,0,-52,0201061bff3906cafb0114fff6ffe20102000000000000...,2402,"[169, 66, -102, 125, -131, -124, 137, -85, 23,...",90,200,tw


# Poli
Capituras no Poli foram feitas a uma distância horizontal constante e logo precisa ser concertada e com o centro **no centro geometrico da antena (entre as antenas 12 e 1)**


In [5]:
poli= pd.DataFrame()

for file in os.listdir("poli"):
    df = single_antenna_to_dataframe("./poli/"+file)
    angle = int(file.split("_")[1])
    distance = int(file.split("_")[3])

    df["angle"] = angle
    df["distance"] = distance if angle == 0 else abs(distance/np.sin(np.deg2rad(angle)))
    poli = pd.concat((poli,df))

poli["capture"] ="poli"
poli

,mac,timestamp,no,rssi,raw,frequency,iq,angle,distance,capture
0,c300000a7a48,2024-10-26T10:43:50.478Z,0,-52,0201061bff3906cafb01140500ecff0201000000000000...,2426,"[137, -69, 53, 143, -150, 22, -1, -151, 157, 1...",120,121.243557,poli
1,c300000a7a48,2024-10-26T10:43:51.001Z,0,-53,0201061bff3906cafb01140500e9ff0201000000000000...,2426,"[-131, -166, 179, -109, 75, 200, -204, 82, -61...",120,121.243557,poli
2,c300000a7a48,2024-10-26T10:43:51.525Z,0,-55,0201061bff3906cafb01140500ecff0201000000000000...,2402,"[65, 157, -164, 32, -16, -170, 162, 2, -21, 17...",120,121.243557,poli
3,c300000a7a48,2024-10-26T10:43:52.048Z,0,-57,0201061bff3906cafb01140500efff0001000000000000...,2480,"[145, 118, -132, 119, -98, -155, 163, -87, 61,...",120,121.243557,poli
4,c300000a7a48,2024-10-26T10:43:52.581Z,0,-57,0201061bff3906cafb01140700efff0201000000000000...,2480,"[-53, -183, 173, -44, 10, 181, -176, -27, 55, ...",120,121.243557,poli
...,...,...,...,...,...,...,...,...,...,...
232,c300000a7a48,2024-10-26T10:02:13.043Z,0,-65,0201061bff3906cafb01140700efff0201000000000000...,2426,"[165, 45, -70, 152, -145, -97, 98, -150, 114, ...",90,420.000000,poli
233,c300000a7a48,2024-10-26T10:02:13.567Z,0,-68,0201061bff3906cafb01140000ecff0201000000000000...,2480,"[-96, 154, -146, -117, 127, -131, 106, 144, -1...",90,420.000000,poli
234,c300000a7a48,2024-10-26T10:02:14.089Z,0,-69,0201061bff3906cafb01140000ecff0201000000000000...,2480,"[77, -198, 201, 93, -109, 199, -179, -115, 148...",90,420.000000,poli
235,c300000a7a48,2024-10-26T10:02:14.623Z,0,-68,0201061bff3906cafb01140007ffec0105000000000000...,2480,"[162, -106, 83, 168, -180, 47, -31, -185, 197,...",90,420.000000,poli


# Camlia

In [6]:
camila= pd.DataFrame()

for file in os.listdir("camila"):
    df = single_antenna_to_dataframe("./camila/"+file)
    angle = int(file.split("_")[1])
    distance = int(file.split("_")[3].split(".")[0])

    df["angle"] = angle
    df["distance"] = distance if angle == 0 else abs(distance/np.sin(np.deg2rad(angle)))
    camila = pd.concat((camila,df))
camila["capture"] ="camila"
camila

,mac,timestamp,no,rssi,raw,frequency,iq,angle,distance,capture
0,c300000a7a46,2024-11-10T18:30:36.550Z,0,-53,0201061bff3906cafb0114f4ffe4ff0701000000000000...,2426,"[-120, 173, -167, -122, 153, -128, 74, 186, -2...",0,150.0,camila
1,c300000a7a46,2024-11-10T18:30:37.083Z,0,-53,0201061bff3906cafb0114fff4ffe20107000000000000...,2402,"[-134, -88, 103, -114, 99, 122, -137, 77, -51,...",0,150.0,camila
2,c300000a7a46,2024-11-10T18:30:37.616Z,0,-60,0201061bff3906cafb0114fff4ffe70107000000000000...,2480,"[-167, -29, 52, -157, 149, 94, -105, 132, -122...",0,150.0,camila
3,c300000a7a46,2024-11-10T18:30:37.877Z,0,-61,0201061bff3906cafb0114f4ffe7ff0701000000000000...,2480,"[-124, 108, -86, -146, 153, -76, 38, 157, -170...",0,150.0,camila
4,c300000a7a46,2024-11-10T18:30:38.411Z,0,-60,0201061bff3906cafb0114f6ffe7ff0501000000000000...,2480,"[-173, -4, 2, -173, 162, 35, -75, 155, -134, -...",0,150.0,camila
...,...,...,...,...,...,...,...,...,...,...
196,c300000a7a46,2024-11-10T17:19:29.949Z,0,-63,0201061bff3906cafb01140200e4ff0a01000000000000...,2402,"[121, 113, -135, 93, -73, -154, 164, -50, 12, ...",90,900.0,camila
197,c300000a7a46,2024-11-10T17:19:30.734Z,0,-67,0201061bff3906cafb01140200e4ff0501000000000000...,2480,"[-36, -188, 188, -16, -54, 184, -188, -83, 93,...",90,900.0,camila
198,c300000a7a46,2024-11-10T17:19:30.995Z,0,-61,0201061bff3906cafb01140005ffe40107000000000000...,2426,"[162, -53, 9, 170, -177, -19, 34, -174, 156, 7...",90,900.0,camila
199,c300000a7a46,2024-11-10T17:19:31.529Z,0,-67,0201061bff3906cafb01140002ffe40107000000000000...,2480,"[-94, -121, 136, -62, 49, 156, -147, 47, -15, ...",90,900.0,camila


# Juntando os datasets

In [7]:
dataset = pd.concat((tw,poli,camila))
dataset

,mac,timestamp,no,rssi,raw,frequency,iq,angle,distance,capture
0,c300000a7a48,2024-10-04T17:03:27.558Z,0,-55,0201061bff3906cafb01140000ffef0102000000000000...,2402,"[94, -140, 123, 116, -137, 97, -71, -157, 158,...",0,100,tw
1,c300000a7a48,2024-10-04T17:03:28.091Z,0,-57,0201061bff3906cafb01140000ffe90100000000000000...,2426,"[-128, -79, 107, -107, 93, 114, -134, 59, -37,...",0,100,tw
2,c300000a7a48,2024-10-04T17:03:28.614Z,0,-57,0201061bff3906cafb01140000ffec0102000000000000...,2426,"[107, 103, -122, 86, -68, -138, 140, -44, 21, ...",0,100,tw
3,c300000a7a48,2024-10-04T17:03:29.148Z,0,-59,0201061bff3906cafb0114fffeffec0102000000000000...,2480,"[-3, 185, -180, -18, 42, -180, 166, 73, -107, ...",0,100,tw
4,c300000a7a48,2024-10-04T17:03:29.409Z,0,-59,0201061bff3906cafb0114feffecff0201000000000000...,2480,"[-175, -39, 79, -163, 159, 106, -114, 147, -11...",0,100,tw
...,...,...,...,...,...,...,...,...,...,...
196,c300000a7a46,2024-11-10T17:19:29.949Z,0,-63,0201061bff3906cafb01140200e4ff0a01000000000000...,2402,"[121, 113, -135, 93, -73, -154, 164, -50, 12, ...",90,900.0,camila
197,c300000a7a46,2024-11-10T17:19:30.734Z,0,-67,0201061bff3906cafb01140200e4ff0501000000000000...,2480,"[-36, -188, 188, -16, -54, 184, -188, -83, 93,...",90,900.0,camila
198,c300000a7a46,2024-11-10T17:19:30.995Z,0,-61,0201061bff3906cafb01140005ffe40107000000000000...,2426,"[162, -53, 9, 170, -177, -19, 34, -174, 156, 7...",90,900.0,camila
199,c300000a7a46,2024-11-10T17:19:31.529Z,0,-67,0201061bff3906cafb01140002ffe40107000000000000...,2480,"[-94, -121, 136, -62, 49, 156, -147, 47, -15, ...",90,900.0,camila


In [8]:
#iq_sample = dataset.sample()["iq"].iloc[0]

def get_data(iq_sample,key:str):
    #compile the IQ data
    iq = np.array([np.angle(complex(iq_sample[i],iq_sample[i+1])) for i in range(0,len(iq_sample),2)])

    # split the reference samples
    ref = iq[:8]

    #split each antenna sample
    a1 = iq[8:-1:3]
    a2 = iq[9::3]
    a3 = iq[10::3]

    #phase diff between each antenna
    a2a1 = pm(a2-a1)
    a3a2 = pm(a3-a2)
    a3a1 = pm(a3-a1)

    dic = {"ref":ref,"a1":a1,"a2":a2,"a3":a3,"a2a1":a2a1,"a3a2":a3a2,"a3a1":a3a1}

    return dic[key]

dataset["ref"] = dataset["iq"].apply(get_data,key="ref")

dataset["a1"] = dataset["iq"].apply(get_data,key="a1")
dataset["a2"] = dataset["iq"].apply(get_data,key="a2")
dataset["a3"] = dataset["iq"].apply(get_data,key="a3")

dataset["a2a1"] = dataset["iq"].apply(get_data,key="a2a1")
dataset["a3a2"] = dataset["iq"].apply(get_data,key="a3a2")
dataset["a3a1"] = dataset["iq"].apply(get_data,key="a3a1")

#np.array([a1,a2,a3])

In [9]:
dataset = dataset.reset_index()
dataset.to_json("./data.json")
dataset

,index,mac,timestamp,no,rssi,raw,frequency,iq,angle,distance,capture,ref,a1,a2,a3,a2a1,a3a2,a3a1
0,0,c300000a7a48,2024-10-04T17:03:27.558Z,0,-55,0201061bff3906cafb01140000ffef0102000000000000...,2402,"[94, -140, 123, 116, -137, 97, -71, -157, 158,...",0,100,tw,"[-0.9795042614277752, 0.7561178310516703, 2.52...","[-0.13998744521308112, -2.4725496553992197, 1....","[0.7395587375163315, -1.4674315925719472, 2.56...","[2.1333708862163094, -0.06216047882113043, -2....","[0.8795461827294127, 1.0051180628272725, 0.900...","[1.393812148699978, 1.4052711137508167, 1.4270...","[2.2733583314293906, 2.4103891765780894, 2.327..."
1,1,c300000a7a48,2024-10-04T17:03:28.091Z,0,-57,0201061bff3906cafb01140000ffe90100000000000000...,2426,"[-128, -79, 107, -107, 93, 114, -134, 59, -37,...",0,100,tw,"[-2.588631051595765, -0.7853981633974483, 0.88...","[-1.574096644845384, 2.605863061612166, 0.2778...","[-0.4414290436740871, -2.6716656922716018, 1.3...","[0.9407624172489815, -1.37557467656394, 2.7768...","[1.1326676011712968, 1.0056565532958182, 1.072...","[1.3821914609230685, 1.2960910157076617, 1.426...","[2.5148590620943656, 2.3017475690034805, 2.499..."
2,2,c300000a7a48,2024-10-04T17:03:28.614Z,0,-57,0201061bff3906cafb01140000ffec0102000000000000...,2426,"[107, 103, -122, 86, -68, -138, 140, -44, 21, ...",0,100,tw,"[0.7663528474154319, 2.5275736165818525, -2.02...","[1.7354167422407016, -0.46222310853983833, -2....","[2.884093776595623, 0.6026499934445425, -1.611...","[-2.019331736209071, 2.0542433283620958, -0.17...","[1.1486770343549213, 1.064873101984381, 1.0710...","[1.379759794374893, 1.4515933349175532, 1.4379...","[2.528436828729814, 2.516466436901934, 2.50893..."
3,3,c300000a7a48,2024-10-04T17:03:29.148Z,0,-59,0201061bff3906cafb0114fffeffec0102000000000000...,2480,"[-3, 185, -180, -18, 42, -180, 166, 73, -107, ...",0,100,tw,"[1.5870111217992988, -3.0419240010986313, -1.3...","[1.0768549578753155, -1.2649174553900444, 2.87...","[2.402457752616702, 0.28297689337281756, -1.95...","[-2.356194490192345, 1.6994928280729107, -0.44...","[1.3256027947413864, 1.547894348762862, 1.4612...","[1.524533064370539, 1.416515934700093, 1.50892...","[2.850135859111926, 2.9644102834629553, 2.9701..."
4,4,c300000a7a48,2024-10-04T17:03:29.409Z,0,-59,0201061bff3906cafb0114feffecff0201000000000000...,2480,"[-175, -39, 79, -163, 159, 106, -114, 147, -11...",0,100,tw,"[-2.922318747112096, -1.119493769638315, 0.588...","[3.011890116433881, 0.6588060361174762, -1.617...","[-1.9843384163437636, 2.0431646449493255, -0.2...","[-0.5176491093767744, -2.7221151918344324, 1.3...","[1.2869567744019417, 1.3843586088318491, 1.410...","[1.4666893069669893, 1.517905470395828, 1.5202...","[2.753646081368931, 2.9022640792276775, 2.9307..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,196,c300000a7a46,2024-11-10T17:19:29.949Z,0,-63,0201061bff3906cafb01140200e4ff0a01000000000000...,2402,"[121, 113, -135, 93, -73, -154, 164, -50, 12, ...",90,900.0,camila,"[0.7512234397812532, 2.538362800665108, -2.013...","[3.0681055933022057, 0.83798122500839, -1.1690...","[0.7266423406817256, -1.4254863161591944, 2.77...","[-2.1995926132103296, 2.114383715095553, -0.16...","[-2.34146325262048, -2.2634675411675844, -2.33...","[-2.926234953892055, -2.7433152759248385, -2.9...","[1.015487100667051, 1.276402490087163, 1.00813..."
19302,197,c300000a7a46,2024-11-10T17:19:30.734Z,0,-67,0201061bff3906cafb01140200e4ff0501000000000000...,2480,"[-36, -188, 188, -16, -54, 184, -188, -83, 93,...",90,900.0,camila,"[-1.7599953488948648, -0.08490179344972196, 1....","[1.5218159258711694, -0.8359878721668329, -2.9...","[-1.215565464748215, -2.9912212255826587, 1.10...","[2.5424173306034863, 0.28379410920832787, -2.0...","[-2.7373813906193845, -2.155233353415826, -2.2...","[-2.525202511827885, -3.0081699723885995, 3.12...","[1.0206014047323169, 1.1197819813751608, 0.923..."
19303,198,c300000a7a46,2024-11-10T17:19:30.995Z,0,-61,0201061bff3906cafb01140005ffe40107000000000000...,2